In [3]:
import glob
from concurrent.futures import ProcessPoolExecutor, as_completed
import torch
from osgeo import gdal, ogr
from Functions import Prediction
from multiprocessing import Manager
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

In [ ]:
if __name__ == "__main__":
    R1 = glob.glob(rf'./ImageDirectory/**/*.tif', recursive=True)
    OP_D = 'Name of Mosiac Raster'
    Output = './OutputPath'
    Mosaic_Path = 'Location to your output Mosaic'
    print(len(R1))
    model_chkpt = rf'/CheckPoint/version_4/checkpoints/Efficentnet_b7_Imagenet.ckpt'
    available_devices = [0,2]  # List of available CUDA devices
    manager = Manager()
    device_queue = manager.Queue()
    for device in available_devices:
        device_queue.put(device)

    args = [(R1[i], model_chkpt, rf'{Output}/{OP_D}', device_queue) for i in range(len(R1))]
    
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(Prediction, arg) for arg in args]
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error in process: {e}")
                
    R2 = glob.glob(rf'{Output}/{OP_D}/*.tif')
    gdal.BuildVRT(rf'{OP_D}.vrt', R2)
    options = ["COMPRESS=LZW"]
    mosaic_ds = gdal.Translate(rf'{Mosaic_Path}/{OP_D}.tif', rf'{OP_D}.vrt', creationOptions=options)
    mosaic_ds = None